Based on:
https://github.com/ml4a/ml4a-guides/blob/master/notebooks/recurrent_neural_networks.ipynb

In [112]:
import os

#if using Theano with GPU
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

import random
import numpy as np
from glob import glob
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout

In [227]:
# Based on these ideas, let us create a generate function
def generate(temperature=0.35, seed=None, num_chars=100):
    predict_more=lambda x: len(x) < num_chars
    
    if seed is not None and len(seed) < max_len:
        raise Exception('Seed text must be at least {} chars long'.format(max_len))

    # if no seed text is specified, randomly select a chunk of text
    else:
        start_idx = random.randint(0, len(text) - max_len - 1)
        seed = text[start_idx:start_idx + max_len]

    sentence = seed
    generated = sentence

    while predict_more(generated):
        # generate the input tensor
        # from the last max_len characters generated so far
        x = np.zeros((1, max_len, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_labels[char]] = 1.

        # this produces a probability distribution over characters
        probs = model.predict(x, verbose=0)[0]

        # sample the character to use based on the predicted probabilities
        next_idx = sample(probs, temperature)
        next_char = labels_char[next_idx]

        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

In [228]:
# A function to draw samples from a Boltzmann distribution
def sample(probs, temperature):
    """samples an index from a vector of probabilities
    (this is not the most efficient way but is more robust)"""
    a = np.log(probs)/temperature
    dist = np.exp(a)/np.sum(np.exp(a))
    choices = range(len(probs))
    return np.random.choice(choices, p=dist)

In [226]:
# load up models and text files
model_file_json = "direction_text_gen_model.json"
model_file_param = "direction_text_gen_model_params_50.h5"
text_files = glob('../data/directions.txt')
text_files[0:5]
text = '\n'.join([open(f, 'r').read() for f in text_files])
text[0:500]
chars = list(set(text))
max_len = 20
char_labels = {ch:i for i, ch in enumerate(chars)}
labels_char = {i:ch for i, ch in enumerate(chars)}

In [229]:

from keras.models import model_from_json
# load json and create model
json_file = open(model_file_json, 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)
# load weights into new model
fname = model_file_param
model.load_weights(fname)
print("Loaded model: " + model_file_json + " " + model_file_param + " from disk")
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Loaded model: direction_text_gen_model.json direction_text_gen_model_params_50.h5 from disk


In [231]:
print('%s' % generate(temperature=0.55, seed='This cake recipe delicious a creamy cake that is easy to make.', num_chars=1000))

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log


ing. Center 8 inch consectioners sugar, coffee, 1/4 cup cocoa, and white sugar. Blend in the water and stir until well blended. Add eggs one at a time, beating well after each addition. Set aside. In a medium bowl, combine flour, sugar, baking powder, salt, and spices. In an ext heat until thick. Repeat ut the batter. Bake in a preheated 350 degrees F (175 degrees C) for 30 to 35 minutes or until cake tests done. Cool. 
Preheat oven to 350 degrees F (175 degrees C) for 30 to 35 minutes. To Make Cool oven top the freezed for 2 hours. Combine with the white sugar and beat until stiff peaks form. Fold 1/3 of the whites into the batter alternately with milk in the bottom of a 9x13 inch pan. In a mixing bowl; stir in a medium lightly flour a bowl. Beat egg whites until stiff. Cover and place in a greased loaf pan. Let rise until doubled in a stiff, and stir. Pour batter into one 9 x 13 inch pan. Bake at 350 degrees F (175 degrees C). Grease and flour two 9 inch round cake pans. Prepare an 
